## Azizu Ahmad Rozaki Riyanto

In [1]:
import pandas as pd

## Dataset from https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp/data

In [2]:
train_df = pd.read_csv("/kaggle/input/sentimen/train.txt",delimiter=';', header=None, names=['sentence','label'])
val_df = pd.read_csv("/kaggle/input/sentimen/val.txt",
                 delimiter=';', header=None, names=['sentence','label'])

tets_df = pd.read_csv("/kaggle/input/sentimen/test.txt",
                 delimiter=';', header=None, names=['sentence','label'])

In [11]:
df = pd.concat([train_df, val_df, tets_df], axis=0)
df = df.reset_index()

In [12]:
df = df.drop(columns='index')

In [13]:
df.tail(10)

,sentence,label
19990,i really feel like they were gentle reminders ...,love
19991,i dont blame it all to them and im not angry a...,love
19992,i feel tortured delilahlwl am considering i ha...,anger
19993,i told my fiance how i am feeling so angry and...,anger
19994,i can feel its suffering,sadness
19995,i just keep feeling like someone is being unki...,anger
19996,im feeling a little cranky negative after this...,anger
19997,i feel that i am useful to my people and that ...,joy
19998,im feeling more comfortable with derby i feel ...,joy
19999,i feel all weird when i have to meet w people ...,fear


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  20000 non-null  object
 1   label     20000 non-null  object
dtypes: object(2)
memory usage: 312.6+ KB


In [15]:
df.isnull().sum()

sentence    0
label       0
dtype: int64

In [16]:
df.iloc[19999,:]

sentence    i feel all weird when i have to meet w people ...
label                                                    fear
Name: 19999, dtype: object

In [17]:
temp = df.duplicated(keep = False)
df[temp]

,sentence,label
4975,i feel more adventurous willing to take risks ...,joy
13846,i feel more adventurous willing to take risks ...,joy


In [18]:
df = df.drop_duplicates()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19999 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  19999 non-null  object
 1   label     19999 non-null  object
dtypes: object(2)
memory usage: 468.7+ KB


In [20]:
unique_values = df.apply(lambda x: x.unique())
print(unique_values)

sentence    [i didnt feel humiliated, i can go from feelin...
label             [sadness, anger, love, surprise, fear, joy]
dtype: object


In [21]:
temp = pd.get_dummies(df.label)
df = pd.concat([df,temp],axis=1)
df = df.drop(columns='label')
df.head()

,sentence,anger,fear,joy,love,sadness,surprise
0,i didnt feel humiliated,False,False,False,False,True,False
1,i can go from feeling so hopeless to so damned...,False,False,False,False,True,False
2,im grabbing a minute to post i feel greedy wrong,True,False,False,False,False,False
3,i am ever feeling nostalgic about the fireplac...,False,False,False,True,False,False
4,i am feeling grouchy,True,False,False,False,False,False


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19999 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  19999 non-null  object
 1   anger     19999 non-null  bool  
 2   fear      19999 non-null  bool  
 3   joy       19999 non-null  bool  
 4   love      19999 non-null  bool  
 5   sadness   19999 non-null  bool  
 6   surprise  19999 non-null  bool  
dtypes: bool(6), object(1)
memory usage: 429.7+ KB


In [23]:
df['sentence'] = df['sentence'].apply(str.lower)
df['sentence']

0                                  i didnt feel humiliated
1        i can go from feeling so hopeless to so damned...
2         im grabbing a minute to post i feel greedy wrong
3        i am ever feeling nostalgic about the fireplac...
4                                     i am feeling grouchy
                               ...                        
19995    i just keep feeling like someone is being unki...
19996    im feeling a little cranky negative after this...
19997    i feel that i am useful to my people and that ...
19998    im feeling more comfortable with derby i feel ...
19999    i feel all weird when i have to meet w people ...
Name: sentence, Length: 19999, dtype: object

In [24]:
# ! pip install nltk


In [25]:
feature = df['sentence'].values
label = df[['anger','fear','joy','love','sadness','surprise']]

In [26]:
import nltk
from nltk.stem import PorterStemmer
import numpy as np

stemmer = PorterStemmer()
feature = np.vectorize(stemmer.stem)(feature)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [27]:
feature

array(['i didnt feel humili',
       'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awak',
       'im grabbing a minute to post i feel greedy wrong', ...,
       'i feel that i am useful to my people and that gives me a great feeling of achiev',
       'im feeling more comfortable with derby i feel as though i can start to step out my shel',
       'i feel all weird when i have to meet w people i text but like dont talk face to face w'],
      dtype='<U300')

In [28]:
len(label)

19999

In [57]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(feature,label,test_size=0.2,random_state=42,shuffle=True,stratify = label)

In [58]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

token = Tokenizer(num_words=10000)
token.fit_on_texts(x_train)
token.fit_on_texts(x_val)

print(len(token.word_index))

sekuens_train = token.texts_to_sequences(x_train)
sekuens_val = token.texts_to_sequences(x_val)

pad_train = pad_sequences(sekuens_train,maxlen =50)
pad_val = pad_sequences(sekuens_val,maxlen =50)

18065


In [59]:
print(len(pad_train_normalized))


13999


In [64]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adamax
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
import os

learning_rate = 0.001

model = tf.keras.Sequential([
tf.keras.layers.Embedding(input_dim=10000, output_dim=64,input_length=50,trainable =True),
tf.keras.layers.LSTM(256),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(1024, activation='relu'),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(6, activation='softmax')
])

for layer in model.layers :
    layer.trainable = True

model.compile(loss='categorical_crossentropy',optimizer=Adamax(learning_rate),metrics=['accuracy'])

In [65]:
num_epochs = 50
batch_size = 8
folder_path = f"run/"

callbacks = [
            tf.keras.callbacks.ModelCheckpoint(os.path.join(folder_path, f"best_modelfold10.h5"), save_best_only=True, verbose=1),
            tf.keras.callbacks.EarlyStopping(patience=100, monitor='val_loss', verbose=1),
            tf.keras.callbacks.TensorBoard(log_dir='logs')
            ]

history = model.fit(pad_train, y_train, epochs=num_epochs,
                    validation_data=(pad_val, y_val), verbose=2,batch_size=batch_size,callbacks=callbacks,)


Epoch 1/50

Epoch 1: val_loss improved from inf to 1.50470, saving model to run/best_modelfold10.h5
2000/2000 - 30s - loss: 1.4796 - accuracy: 0.4212 - val_loss: 1.5047 - val_accuracy: 0.3920 - 30s/epoch - 15ms/step
Epoch 2/50


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 1.50470
2000/2000 - 13s - loss: 0.9967 - accuracy: 0.6384 - val_loss: 1.9447 - val_accuracy: 0.5428 - 13s/epoch - 7ms/step
Epoch 3/50

Epoch 3: val_loss did not improve from 1.50470
2000/2000 - 13s - loss: 0.6727 - accuracy: 0.7673 - val_loss: 2.1030 - val_accuracy: 0.5395 - 13s/epoch - 6ms/step
Epoch 4/50

Epoch 4: val_loss improved from 1.50470 to 1.19582, saving model to run/best_modelfold10.h5
2000/2000 - 13s - loss: 0.5011 - accuracy: 0.8247 - val_loss: 1.1958 - val_accuracy: 0.7125 - 13s/epoch - 7ms/step
Epoch 5/50

Epoch 5: val_loss did not improve from 1.19582
2000/2000 - 13s - loss: 0.4047 - accuracy: 0.8617 - val_loss: 2.9913 - val_accuracy: 0.4190 - 13s/epoch - 6ms/step
Epoch 6/50

Epoch 6: val_loss improved from 1.19582 to 0.78790, saving model to run/best_modelfold10.h5
2000/2000 - 12s - loss: 0.3182 - accuracy: 0.8921 - val_loss: 0.7879 - val_accuracy: 0.7653 - 12s/epoch - 6ms/step
Epoch 7/50

Epoch 7: val_loss did not improve from 

In [66]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/working/run/best_modelfold10.h5')


train_scores = model.evaluate(pad_train, y_train)
print(f'Training Accuracy: {train_scores[1]:.4f}')

# Evaluate on validation data
val_scores = model.evaluate(pad_val, y_val)
print(f'Validation Accuracy: {val_scores[1]:.4f}')

500/500 [==============================] - 2s 4ms/step - loss: 0.0385 - accuracy: 0.9856
Training Accuracy: 0.9856
125/125 [==============================] - 0s 4ms/step - loss: 0.7736 - accuracy: 0.8428
Validation Accuracy: 0.8428
